# Imports

In [1]:
import sys
import logging
import shutil
from uuid import uuid4
from tempfile import gettempdir
from os.path import join, isdir
import folium
from aequilibrae import Parameters, logger, Project
import pandas as pd

# Model place (Country) to build the model for

In [2]:
model_place = 'Andorra'

In [3]:
fldr = f'../../{model_place}'

# Open the model

In [4]:
project = Project()
project.open(fldr)

# MOVE ALL CODE BELOW TO A FUNCTION
# THIS NOTEBOOK SHOULD CONTAIN THE BUILD OF THE DEMAND MODEL WITHIN THE PROJECT AS WELL

# Adds fields missing in the data

In [5]:
speeds_and_capacities = pd.read_csv('../model/network/speeds_and_capacities.csv')

### Number of lanes and lane_capacity

In [6]:
project.conn.execute('Update link_types set lanes=1, lane_capacity=400 where link_type_id not in ("z", "y")')
project.conn.commit()
for idx, rec in speeds_and_capacities.iterrows():
    project.conn.execute('Update link_types set lanes=?, lane_capacity=? where link_type=?', [rec.lanes, rec.lane_capacity, rec.link_type])
    project.conn.execute('Update links set lanes_ab=? where lanes_ab is NULL and direction>=0 and link_type=?', [rec.lanes, rec.link_type])
    project.conn.execute('Update links set lanes_ba=? where lanes_ba is NULL and direction>=0 and link_type=?', [rec.lanes, rec.link_type])
    
project.conn.execute('Update links set lanes_ab = (select lanes from link_types where link_type=links.link_type) where lanes_ab is NULL')
project.conn.execute('Update links set lanes_ba =(select lanes from link_types where link_type=links.link_type) where lanes_ba is NULL')

### Speeds

In [7]:
for idx, rec in speeds_and_capacities.iterrows():
    project.conn.execute('Update links set speed_ab=? where direction>=0 and link_type=?', [rec.speed, rec.link_type])
    project.conn.execute('Update links set speed_ba=? where direction<1 and link_type=?', [rec.speed, rec.link_type])

# Fill the missing ones
project.conn.execute('Update links set speed_ab=40 where direction>=0 and speed_ab is NULL')
project.conn.execute('Update links set speed_ba=40 where direction<1 and speed_ba is NULL')
project.conn.commit()

### Link capacities

In [8]:
project.conn.execute('Update links set capacity_ab=lanes_ab* (select lane_capacity from link_types where link_type=links.link_type)')
project.conn.execute('Update links set capacity_ba=lanes_ba* (select lane_capacity from link_types where link_type=links.link_type)')
project.conn.execute("Update links set capacity_ab=0.1 where modes like '%c%' and capacity_ab=0")
project.conn.execute("Update links set capacity_ba=0.1 where modes like '%c%' and capacity_ba=0")

In [ ]:
project.close()